In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Documents/REx_Lab/tinympc-julia/state_constraints/Project.toml`


In [2]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using MeshCat
using ColorTypes
using GeometryBasics: HyperSphere, HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf


include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [3]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
gravity = 9.81
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
# kt=1
km = kt*thrustToTorque #4.4733e-8

# h = 1/500
h = 1/20

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
@show uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# @show uhover = (m*gravity/4)*ones(4) # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * gravity) / kt) / 4) * ones(4) = [0.5833333520642209, 0.5833333520642209, 0.5833333520642209, 0.5833333520642209]


In [4]:
function quad_dynamics(x,u)
  r = x[1:3]
  q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
  v = x[8:10]
  ω = x[11:13]
  Q = qtoQ(q)
  
  ṙ = v
  q̇ = 0.5*L(q)*H*ω
  
  b = 1/m

  v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 

  ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
  
  return [ṙ; q̇; v̇; ω̇]
end
function quad_dynamics_rk4(x,u)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5*h*f1, u)
  f3 = quad_dynamics(x + 0.5*h*f2, u)
  f4 = quad_dynamics(x + h*f3, u)
  xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
  xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [5]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [6]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

function TransformPlane(visObject, center, dir)

    tol = 1e-4
    x_axis = [1.0; 0.0; 0.0]

    # Compute axis angle
    dot = x_axis'*dir
    if dot > 1-tol
        axis = x_axis
    elseif dot < -1+tol
        axis = -x_axis
    else
        axis = cross(x_axis, dir)
    end
    angle = acos(x_axis'*dir)

    settransform!(visObject, Translation(center...) ∘ LinearMap(AngleAxis(angle, axis...)))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /Users/anoushkaalavill/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


TransformPlane (generic function with 1 method)

In [7]:
Tfinal = 15.0
N = Int(Tfinal/h)+1

# Cost weights
max_dev_x = [0.1; 0.1; 0.1; 0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
max_dev_u = [0.5; 0.5; 0.5; 0.5]
Q = spdiagm(1 ./(max_dev_x.^2))
R = spdiagm(1 ./(max_dev_u.^2))
Qf = Q

# # Number of LQR direct solve matrix sets to cache
ρ_list_length = 1

# Allocate cache
cache = (
    A = Ãdyn,
    B = B̃dyn,
    # ρ_list = [diagm(ones(nu)) for _ in 1:ρ_list_length],
    ρ_list = [zeros(1) for _ in 1:ρ_list_length],
    Kinf_list = [zeros(nu,nx) for _ in 1:ρ_list_length],
    Pinf_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    Quu_inv_list = [zeros(nu,nu) for _ in 1:ρ_list_length],
    AmBKt_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    coeff_d2p_list = [zeros(nx,nu) for _ in 1:ρ_list_length],
)

# Precompute all cached matrices for multiple ρ values starting at ρ = .1 and multiplying by 5
for k = 1:ρ_list_length
    # ρ
    # ρ = diagm([1; 1; 1; 1])*1*5^(k-1)
    ρ = 5*5^(k-1);
    R̃ = R + ρ*I;
    Q̃ = Q + ρ*I;
    Q̃f = Qf + ρ*I;
    
    # Compute Kinf, Pinf
    Nriccati = 10000
    P = [zeros(nx,nx) for i = 1:Nriccati];   # cost to go quadratic term
    K = [zeros(nu,nx) for i = 1:Nriccati-1]; # feedback gain
    P[Nriccati] .= Q̃f;
    for k = (N-1):-1:1
        K[k] = (R̃ + cache.B'*P[k+1]*cache.B)\(cache.B'*P[k+1]*cache.A);
        P[k] = Q̃ + cache.A'*P[k+1]*(cache.A - cache.B*K[k]);
    end

    # Cache precomputed values
    cache.ρ_list[k] .= ρ;
    cache.Kinf_list[k] .= K[1];
    cache.Pinf_list[k] .= P[1];
    cache.Quu_inv_list[k] .= (R̃ + cache.B'*cache.Pinf_list[k]*cache.B)\I;
    cache.AmBKt_list[k] .= (cache.A - cache.B*cache.Kinf_list[k])';
    cache.coeff_d2p_list[k] .= cache.Kinf_list[k]'*R̃ - cache.AmBKt_list[k]*cache.Pinf_list[k]*cache.B;
end

# Create reference sketch trajectory
X̃ref = [zeros(nx) for i = 1:N]
Xref = [[0; 0; .5; 1; zeros(9)] for i = 1:N]
Uref = [zeros(nu) for i = 1:N-1]

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))
;

In [8]:
N

301

In [9]:
cache.ρ_list

1-element Vector{Vector{Float64}}:
 [5.0]

In [10]:
function export_mat_to_c(declare, data)
    str = "tinytype" * declare * "= {\n"
    for i = 1:size(data, 1)
        str = str * "  "
        for j = 1:size(data, 2)
            if i == size(data, 1) && j == size(data, 2)
                this_str = @sprintf("%.7f\t", data[i, j])
            else
                this_str = @sprintf("%.7f,\t", data[i, j])
            end
            str = str * this_str
        end
        str = str * "\n"
    end
    str = str * "};"
    return str
end

function export_Xref_to_c(declare, data)
    str = "tinytype" * declare * "= {\n"
    for i = 1:size(data, 1)
        str = str * "  "
        for j = 1:nx
            if i == size(data, 1) && j == nx
                this_str = @sprintf("%.7f\t", data[i][j])
            else
                this_str = @sprintf("%.7f,\t", data[i][j])
            end
            # str = str * this_str * "f"
            str = str * this_str
        end
        str = str * "\n"
    end
    str = str * "};"
    return str
end

export_Xref_to_c (generic function with 1 method)

In [11]:
cache.Quu_inv_list[1]

4×4 Matrix{Float64}:
  0.0321638    -0.000415222   0.0319393    -0.000376201
 -0.000415222   0.0320971    -0.000320613   0.0319504
  0.0319393    -0.000320613   0.031973     -0.000280095
 -0.000376201   0.0319504    -0.000280095   0.0320176

In [12]:
print("tinytype rho_value = " * string(cache.ρ_list[1][1]) * ";\n\n")

print(export_mat_to_c(" Adyn_data[NSTATES*NSTATES] ", cache.A) * "\n\n")
print(export_mat_to_c(" Bdyn_data[NSTATES*NINPUTS] ", cache.B) * "\n\n")

print(export_mat_to_c(" Kinf_data[NINPUTS*NSTATES] ", cache.Kinf_list[1]) * "\n\n")
print(export_mat_to_c(" Pinf_data[NSTATES*NSTATES] ", cache.Pinf_list[1]) * "\n\n")
print(export_mat_to_c(" Quu_inv_data[NINPUTS*NINPUTS] ", cache.Quu_inv_list[1]) * "\n\n")
print(export_mat_to_c(" AmBKt_data[NSTATES*NSTATES] ", cache.AmBKt_list[1]) * "\n\n")
print(export_mat_to_c(" coeff_d2p_data[NSTATES*NINPUTS] ", cache.coeff_d2p_list[1]) * "\n\n")

print(export_mat_to_c(" Q_data[NSTATES*NSTATES]", Q) * "\n\n")
print(export_mat_to_c(" Qf_data[NSTATES*NSTATES]", Qf) * "\n\n")
print(export_mat_to_c(" R_data[NINPUTS*NINPUTS]", R) * "\n\n")


# print(export_Xref_to_c(" Xref_data[NTOTAL*NSTATES] ", X̃ref))

tinytype rho_value = 5.0;

tinytype Adyn_data[NSTATES*NSTATES] = {
  1.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0245250,	0.0000000,	0.0500000,	0.0000000,	0.0000000,	0.0000000,	0.0002044,	0.0000000,	
  0.0000000,	1.0000000,	0.0000000,	-0.0245250,	0.0000000,	0.0000000,	0.0000000,	0.0500000,	0.0000000,	-0.0002044,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	1.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0500000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	0.0000000,	1.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0250000,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	0.0000000,	0.0000000,	1.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0250000,	0.0000000,	
  0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	1.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0250000,	
  0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.9810000,	0.0000000,	1.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0122625

tinytype Q_data[NSTATES*NSTATES]= {
  100.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000,	100.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	100.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	0.0000000,	4.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	0.0000000,	0.0000000,	4.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	400.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	4.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	0.0000000,	
  0.0000000

In [13]:
# display(cache.Kinf)
# display(cache.Pinf)
# display(cache.Quu_inv)
# display(cache.AmBKt)
# display(cache.coeff_d2p)

cache_size = 4*( *(size(cache.Kinf_list)...) + *(size(cache.Pinf_list)...) + *(size(cache.Quu_inv_list)...) + *(size(cache.AmBKt_list)...) + *(size(cache.coeff_d2p_list)...) )

20

In [14]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))



# Clean up MeshCat environment from previous run
for k in 1:10000
    delete!(vis["xlin " * string(k)])
    delete!(vis["linearized constraint " * string(k)])
    delete!(vis["obstacle " * string(k)])
    delete!(vis["bounding obstacle " * string(k)])
end



# Used for skipping skip_obs number of linearized obstacle constraints (skips both visualization and computation)
skip_obs = 1

# MPC setup
Nh = 20


# Create obstacle in MeshCat
obstacle_center = [0.0; 2.0; 0.5]
obstacle_velocity = 1 * h

r_vis = 0.1
r_obstacle = 0.75

# Create visualization data to populate during MPC solution

bounding_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
thin_rect_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.25))
constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(-0.0005,-0.5,-0.5), Vec(0.001,1,1)), thin_rect_material) for k in 1:skip_obs:Nh]


# Instantiate TinyMPC-ADMM variables

x = [zeros(nx) for i = 1:Nh]
u = [zeros(nu) for i = 1:Nh-1]

q = [zeros(nx) for i = 1:Nh];
r = [zeros(nu) for i = 1:Nh-1];

p = [zeros(nx) for i = 1:Nh];      # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(nu) for i = 1:Nh-1];    # feedforward control

# Input constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5

# State constraints
# xmin = [-Inf .* ones(nx) for i = 1:Nh]
# xmax = [[Inf; 0.5; Inf .* ones(10)] for i = 1:Nh]
xmin = [-Inf for i = 1:Nh] # Only one -Inf per knot point because only one inequality constraint (for now)
xmax = [zeros(1) for i = 1:Nh]

# Aineq = [speye(nx) for i = 1:Nh]
Aineq = [zeros(nx) for i = 1:Nh]

for k = 1:skip_obs:Nh
    xc = obstacle_center - X̃ref[k][1:3]
    a = xc/norm(xc)

    Aineq[k][1:3] = a'

    q_c = obstacle_center - r_obstacle*a
    b = a'*q_c
    xmax[k][1] = b
    TransformPlane(constraintPlanes[cld(k,skip_obs)], q_c, a)
end



# Simulate
Nsim = N - Nh
xhist = [zeros(nx+1) for _ = 1:Nsim] # nx+1 because not reduced form
xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ = 1:Nsim-1]
@time for k = 1:Nsim-1 # Nsim-1 knotpoints for control, Nsim knotpoints for state

    # State slack (auxiliary) and dual variables
    v = [zeros(nx) for i = 1:Nh]
    vnew = [zeros(nx) for i = 1:Nh]
    g = [zeros(nx) for i = 1:Nh]

    # Input slack (auxiliary) and dual variables
    z = [zeros(nu) for i = 1:Nh-1]
    znew = [zeros(nu) for i = 1:Nh-1]
    y = [zeros(nu) for i = 1:Nh-1]

    x[1] = [xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]]

    # params.constraint_A .= Aineq
    # params.umin .= umin
    # params.umax .= umax
    # params.xmin .= xmin
    # params.xmax .= xmax
    # params.Xref .= X̃ref[k:k+Nh-1]
    # params.Uref .= Uref[k:k+Nh-2]
    
    params = (
        N = Nh,
        Q = Q, R = R, Qf = Qf,
        constraint_A = Aineq,
        umin = umin, umax = umax,
        xmin = xmin, xmax = xmax,
        Xref = X̃ref[k:k+Nh-1], Uref = Uref[k:k+Nh-2], 
        cache = cache,

        ρ_index = [1]
    )

    # Solve
    Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; abs_tol=1e-3, max_iter=500)
    # @show iter, status
    # display("iter: " * string(iter))
    
    # Roll out x_k+1 with actual dynamics
    uhist[k] = Δu[1] + uhover
    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])
    
    # Roll out with real dynamics
    x_real = [zeros(nx+1) for _ = 1:Nh]
    x_real[1] = xhist[k]
    for j = 1:Nh-1
        x_real[j+1] = quad_dynamics_rk4(x_real[j], uhist[k])
    end

    # Visualize solution x
    x_linearized = [Point(xp[1], xp[2], xp[3]+0.01) for xp in x_real]
    if k > 1
        x_lin_name_prev = "xlin " * string(k-1)
        setvisible!(vis[x_lin_name_prev], false)
    end
    x_lin_name = "xlin " * string(k)
    setobject!(vis[x_lin_name], Object(PointCloud(x_linearized), 
        LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    # Create constraints for each knot point in the horizon
    #   assuming the obstacle's center moves at constant velocity between each step
    obstacle_to_quad = x_real[1][1:3] - obstacle_center
    obstacle_to_quad_dir = obstacle_to_quad / norm(obstacle_to_quad)
    obstacle_dist_traveled = obstacle_velocity * obstacle_to_quad_dir
    
    # Update linearized obstacle constraints
    if k > 1
        for i = 1:Nh
            setvisible!(vis["linearized constraint " * string((k-2)*Nh + i)], false)
        end
    end
    constraintPlanes = [setobject!(vis["linearized constraint " * string((k-1)*Nh + i)], HyperRectangle(Vec(-0.0005,-0.5,-0.5), Vec(0.001,1,1)), thin_rect_material) for i in 1:skip_obs:Nh]
    for j = 1:skip_obs:Nh
        obstacle_center_prediction = obstacle_center + obstacle_dist_traveled*j
        xc = obstacle_center_prediction - x_real[j][1:3]
        a = xc/norm(xc) # hyperplane projection expects normalized a

        Aineq[j][1:3] = a'

        q_c = obstacle_center_prediction - r_obstacle*a
        b = a'*q_c
        xmax[j][1] = b
        TransformPlane(constraintPlanes[cld(j,skip_obs)], q_c, a)
    end

    # Update actual obstacle location at x=0
    obstacle_center += obstacle_dist_traveled
    
    # # Visualize cylinder
    # if k > 1
    #     setvisible!(vis["obstacle " * string(k-1)], false)
    #     setvisible!(vis["bounding obstacle " * string(k-1)], false)
    # end
    # cyl_vis = setobject!(vis["obstacle " * string(k)], Cylinder(Point(0, 0, -0.5), Point(0, 0, 0.5), r_vis))
    # cyl_avoid = setobject!(vis["bounding obstacle " * string(k)], Cylinder(Point(0, 0, -5.05), Point(0, 0, 5.05), r_obstacle), bounding_material)
    # settransform!(cyl_vis, Translation(obstacle_center[1], obstacle_center[2], obstacle_center[3]))

    # Visualize sphere
    if k > 1
        setvisible!(vis["obstacle " * string(k-1)], false)
        setvisible!(vis["bounding obstacle " * string(k-1)], false)
    end
    setobject!(vis["obstacle " * string(k)], HyperSphere(Point(obstacle_center...), r_vis))
    setobject!(vis["bounding obstacle " * string(k)], HyperSphere(Point(obstacle_center...), r_obstacle), bounding_material)
end

# delete!(vis["xHistline"])
# xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
# setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
# LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

0.0
3.5541712373394017
0.14457955927296728
1.2914538001511882

0.0
0.8632235636403696
0.0
0.9492941820349787

0.0
0.3159949890042796
0.0
0.24577980626112883

0.0
0.11887907806416909
0.0
0.09953400856747063

0.0
0.04926532554885363
0.0
0.04809437506762425

0.0
0.021737724263905904
0.0
0.02554579431915307

0.0
0.00993870849940924
0.0
0.013206509984494512

0.0
0.004667424404367106
0.0
0.006767342331824543

0.0
0.0023281371442651633
0.0
0.0034678243700759365

0.0
0.0011882035199994867
0.0
0.0017847335412860335



0.0
0.006578670519246793
0.0
0.004933530776883299

0.0
0.0022671854715119663
0.0
0.0018810097421378957



0.0
0.006126494730796812
0.0
0.0049930358512728645

0.0
0.0022779369376206213
0.0
0.002016740798813088

0.0
0.005114008785286443
0.0
0.004343695488331356

0.0
0.001979000127735782
0.0
0.0017937532000877887

0.0
0.004151180346090976
0.0
0.0036198128531100826

0.0
0.0016455474741471597
0.0
0.0015161455432605164

0.0
0.0033374581620453266
0.0
0.002943294821476372

0.0
0.0013354171725260544
0.0
0.0012461511063188713

0.0
0.0026386811403288253
0.0
0.002349121256571829

0.0
0.0010640871045157485
0.0
0.0010033879978382787

0.04512671764580547
0.0031406526932477954
0.0
0.0019333373059404746

0.045009036702852334
0.0023710085941994435
0.0
0.0008589802856623363

0.044899937910850854
0.00220720274824587
0.0
0.00043469426874498396

0.044792794885725004
0.0021668089892112993
0.0
0.0002429984305224453

0.04468621255086724
0.0021530746736405358
0.0
0.0001513041619499722

0.04457989914779232
0.00214483570899988
0.0
0.00010602710718954356

0.04447380576424334
0.002137698644958442
0.0
8.31823245900365e-

0.0
4.413447646421653e-5

0.032077303632071924
0.001535714783816311
0.0
4.4029337309091365e-5

0.03200088762184341
0.001532056334282543
0.0
4.392444862194589e-5

0.031924653653307926
0.0015284066000711277
0.0
4.381980980405764e-5

0.03184860129279586
0.0015247655604205823
0.0
4.371542026197772e-5

0.031772730107672315
0.0015211331946186901
0.0
4.361127940145926e-5

0.03169703966633275
0.0015175094819935842
0.0
4.35073866306146e-5

0.031621529538202804
0.0015138944019621459
0.0
4.340374135644587e-5

0.03154619929373128
0.0015102879339396608
0.0
4.33003429915757e-5

0.031471048504389844
0.0015066900573896397
0.0
4.319719094786345e-5

0.03139607674267417
0.0015031007518893566
0.0
4.309428463650927e-5

0.031321283582095495
0.0014995199969669928
0.0
4.299162347346647e-5

0.031246668597182788
0.001495947772293879
0.0
4.288920687451486e-5

0.03117223136347761
0.001492384057543046
0.0
4.2787034257828205e-5

0.031097971457530967
0.0014888288324182639
0.0
4.268510504033124e-5

0.0310238884569093


0.0014782138929602534
0.0
4.2380771998543065e-5

0.030802696677670546
0.0014746924246427512
0.0
4.22798105979838e-5

0.030729317094044327
0.0014711793453386912
0.0
4.217908971166501e-5

0.030656112318597016
0.0014676746350636427
0.0
4.2078608767752435e-5

0.03058308193489121
0.0014641782738598202
0.0
4.1978367194550614e-5

0.030510225527485058
0.0014606902418476397
0.0
4.1878364420572245e-5

0.03043754268192217
0.001457210519172046
0.0
4.177859987831989e-5

0.030365032984736293
0.0014537390860898738
0.0
4.1679072998179745e-5

0.030292696023446107
0.001450275922769903
0.0
4.15797832175116e-5

0.03022053138655174
0.0014468210095563289
0.0
4.148072996864455e-5

0.030148538663535143
0.0014433743268209634
0.0
4.138191268956287e-5

0.03007671744485356
0.0014399358549058508
0.0
4.128333081599572e-5

0.03000506732194383
0.0014365055742947275
0.0
4.1184983789951946e-5

0.02993358788721103
0.0014330834654507907
0.0
4.108687105017911e-5

0.02986227873403461
0.001429669508889557
0.0
4.09889920391

...

0.08418281558883361
0.01061007589768681
0.0
0.004428874634539448

0.08341068438374069
0.010270464169008786
0.0
0.004401814164580187

0.08265449553338444
0.009939996058206946
0.0
0.004375493368839606

0.08191386604044276
0.009618435764984223
0.0
0.004349495272653325

0.08118842244383673
0.009305553481240292
0.0
0.0043238138238752

0.0804778005789179
0.009001125241627772
0.0
0.0042984431083015195

0.07978164534369575
0.00870493277409512
0.0
0.004273377346265955

0.07909961047096586
0.008416763356379198
0.0
0.004248610889385357

0.07843135830616013
0.008136409675271183
0.0
0.0042241382173408

0.07777655959085736
0.007863669690031383
0.0
0.004199953934734685

0.07713489325168421
0.007598346499304576
0.0
0.004176052768074673

0.07650604619459345
0.007340248210255806
0.0
0.004152429562782334

0.07588971310429277
0.00708918781215595
0.0
0.0041290792802953225

0.07528559624877076
0.00684498305305703
0.0
0.004105996995237282

0.07469340528869561
0.006607456318811211
0.0
0.004083177892700851

0.


0.042566587657737964
0.003475491910488737
0.0
0.002314333519003453

0.04247341589586459
0.003468732814815856
0.0
0.002307086759280319

0.04238093386354108
0.0034619279200254205
0.0
0.0022998811482163872

0.04228913075618146
0.0034550793274681912
0.0
0.002292716318360033

0.04219799600183233
0.0034481890799206716
0.0
0.0022855919072647946

0.04210751925553197
0.0034412591630764156
0.0
0.0022785075573864

0.04201769039396053
0.003434291507027065
0.0
0.002271462916010325

0.0419284995100635
0.003427287987708416
0.0
0.0022644576351379953

0.04183993690792698
0.0034202504282843105
0.0
0.0022574913714065725

0.04175199309772659
0.0034131806005294196
0.0
0.0022505637860137373

0.041664658790796505
0.0034060802261293155
0.0
0.0022436745446066664

0.04157792489484666
0.003398950978017734
0.0
0.0022368233172123664

0.041491782509270686
0.0033917944815978207
0.0
0.0022300097781566275

0.041406222920617186
0.003384612316010005
0.0
0.002223233605991859

0.04132123759809819
0.003377406015300788
0.0

0.0027306296383367012
0.0
0.001726480164095101

0.035206716560321344
0.002724216333566676
0.0
0.0017221761724486861

0.035153412073413104
0.0027178230418811733
0.0
0.0017178909768506

0.0351003146587221
0.0027114497762215084
0.0
0.0017136244563556446

0.03504742270444425
0.002705096546371244
0.0
0.0017093764910239284

0.03499473462038408
0.00269876335906305
0.0
0.0017051469619064341

0.03494224883763622
0.002692450218058362
0.0
0.0017009357510425205

0.03488996380801171
0.002686157124260069
0.0
0.0016967427414404934

0.03483787800379565
0.002679884075785788
0.0
0.0016925678170823244

0.03478598991724219
0.0026736310680694486
0.0
0.0016884108628803518

0.034734298060270674
0.002667398093941231
0.0
0.0016842717647119754

0.03468280096403448
0.0026611851437077783
0.0
0.00168015040936359

0.03463149717861286
0.002654992205241846
0.0
0.0016760466845611166

0.034580385272582115
0.002648819264053359
0.0
0.0016719604789253162

0.034529463832766005
0.0026426663033685127
0.0
0.001667891681987887

Excessive output truncated after 524299 bytes.


0.33791030085311546
0.19291603718763373
0.0
0.033238160409536574

0.331895953259986
0.18906628870737907
0.0
0.032557595761272644

0.3260029189197743
0.18522532540523495
0.0
0.03189044528381781

0.3202287325338724
0.18141725568544542
0.0
0.031236581600276175

0.3145709749196445
0.1776572561564549
0.0
0.030595826765630607

0.30902727378730455
0.1739547109239481
0.0
0.029967973837451145

0.3035953038407842
0.17031524275933463
0.0
0.029352799604924068

0.29827278646119415
0.16674202993712695
0.0
0.028750072257167913

0.2930574891382634
0.16323666174048457
0.0
0.0281595560967364

0.28794722475440093
0.15979969544200667
0.0
0.027581014490771147

0.2829398507868302
0.15643101971282825
0.0
0.027014211747315303

0.2780332684682607
0.15313009224138807
0.0
0.026458914321100258

0.2732254219307832
0.14989609542943483
0.0
0.025914891591733852

0.2685142973477088
0.14672803860263484
0.0
0.025381916363609522

0.263897922082072
0.14362482522585318
0.0
0.02485976518142502

0.2593743638467176
0.1405852

In [15]:
X1 = [SVector{13}(x_) for x_ in xhist];
visualize!(vis, quad_model, Tfinal, X1)

In [16]:
# constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(0.0,0.0,0.0), Vec(0.001,2.0,1.0)), thin_rect_material) for k in 1:Nh]
for k in 1:N
    delete!(vis["linearized constraint " * string(k)])
end

# Create MeshCat animation
anim = MeshCat.Animation()

for k = 1:Nsim-1
    atframe(anim, k-1) do
        # Set visibility of each object to false
        if k == 1
            for j = 1:Nsim
                for i = 1:Nh
                    setvisible!(vis["linearized constraint " * string((j-1)*Nh + i)], false)
                end
                setvisible!(vis["xlin " * string(j)], false)
                setvisible!(vis["obstacle " * string(j)], false)
                setvisible!(vis["bounding obstacle " * string(j)], false)
            end
        end

        # Change visible obstacle
        setvisible!(vis["obstacle " * string(k)], true)
        setvisible!(vis["bounding obstacle " * string(k)], true)
        if k > 1
            setvisible!(vis["obstacle " * string(k-1)], false)
            setvisible!(vis["bounding obstacle " * string(k-1)], false)
        end


        # Update obstacle constraint visuals
        for i = 1:Nh
            setvisible!(vis["linearized constraint " * string((k-1)*Nh + i)], true)
        end
        if k > 1
            for i = 1:Nh
                setvisible!(vis["linearized constraint " * string((k-2)*Nh + i)], false)
            end
        end
        

        # Change visible horizon
        setvisible!(vis["xlin " * string(k)], true)
        if k > 1
            setvisible!(vis["xlin " * string(k-1)], false)
        end

        # Update quadrotor transform
        xbar = RBState(quad_model, xhist[k])
        if quad_model.ned
            rq = position(xbar)
            vq = linear_velocity(xbar)
            rq = SA[rq[1],-rq[2],-rq[3]]
            vq = SA[vq[1],-vq[2],-vq[3]]
            xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
        end
        settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))
    end
end

setanimation!(vis, anim)

In [17]:
MeshCat.convert_frames_to_video(
    "/Users/anoushkaalavill/Documents/REx_Lab/tinympc-julia/state_constraints/state_constraints_tinympc.tar")

ErrorException: The output path /Users/anoushkaalavill/Documents/REx_Lab/tinympc-julia/state_constraints/output.mp4 already exists. To overwrite that file, you can pass `overwrite=true` to this function

In [18]:
a = [2.0; 3.0; 5.0]
a = a/norm(a)

3-element Vector{Float64}:
 0.3244428422615251
 0.48666426339228763
 0.8111071056538127

In [19]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))

params = (
    N = Nh,
    Q = Q, R = R, Qf = Qf,
    constraint_A = Aineq,
    umin = umin, umax = umax,
    xmin = xmin, xmax = xmax,
    Xref = X̃ref[1:Nh], Uref = Uref[1:Nh-1], 
    cache = cache,

    ρ_index = [1]
)

# MPC setup
Nh = 20

# Instantiate TinyMPC-ADMM variables
x = [zeros(nx) for i = 1:Nh]
u = [zeros(nu) for i = 1:Nh-1]

q = [zeros(nx) for i = 1:Nh];
r = [zeros(nu) for i = 1:Nh-1];

p = [zeros(nx) for i = 1:Nh];      # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(nu) for i = 1:Nh-1];    # feedforward control

# Input constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5

# State constraints
xmin = [-Inf for i = 1:Nh] # Only one -Inf per knot point because only one inequality constraint (for now)
xmax = [zeros(1) for i = 1:Nh]

# Aineq = [speye(nx) for i = 1:Nh]
Aineq = [zeros(nx) for i = 1:Nh]
# Aineq[1] = [0.32444; 0.48666; 0.81111; zeros(9)]

# xmax[1] = [1]

# State slack (auxiliary) and dual variables
v = [zeros(nx) for i = 1:Nh]
vnew = [zeros(nx) for i = 1:Nh]
g = [zeros(nx) for i = 1:Nh]

# Input slack (auxiliary) and dual variables
z = [zeros(nu) for i = 1:Nh-1]
znew = [zeros(nu) for i = 1:Nh-1]
y = [zeros(nu) for i = 1:Nh-1]

x[1] = params.Xref[1]

# display(x)
Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; abs_tol=1e-3, max_iter=100)
@show iter

Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; abs_tol=1e-3, max_iter=100)
@show iter

# p[params.N] = -params.Qf*params.Xref[params.N]

# for k = 1:(params.N-1)
#     z[k] .= min.(params.umax, max.(params.umin, u[k] + y[k]))
# end
# for k = 1:params.N
#     v[k] .= project_hyperplane(k, vis, x[k] + g[k], params.constraint_A[k], params.xmax[k][1])
# end
# update_linear_cost!(v,g,z,y,p,q,r,params)

# backward_pass_grad!(q, r, p, d, params)
# forward_pass!(d, x, u, params)
# display(params.cache.Quu_inv_list[1])
# display(params.cache.B)
# display(params.cache.AmBKt_list[1])
# display(params.cache.Kinf_list[1])
# display(params.cache.coeff_d2p_list[1])


101